# SpringBoard Capstone3: Coronavirus Tweets Sentiment

    Data from Kaggle.com: https://www.kaggle.com/datatattle/covid-19-nlp-text-classification/metadata
## XGboost

In [1]:
# !pip install wordcloud

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Ling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud
import re
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
tweet_whole_train = pd.read_csv('/Users/Ling/Desktop/01_Springboard/Corona_Tweet/Data/Corona_NLP_train.csv', index_col = 0)
print(tweet_whole_train.info())
test = pd.read_csv('/Users/Ling/Desktop/01_Springboard/Corona_Tweet/Data/Corona_NLP_test.csv', index_col = 0)
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41157 entries, 3799 to 44955
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ScreenName     41157 non-null  int64 
 1   Location       32567 non-null  object
 2   TweetAt        41157 non-null  object
 3   OriginalTweet  41157 non-null  object
 4   Sentiment      41157 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3798 entries, 1 to 3798
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ScreenName     3798 non-null   int64 
 1   Location       2964 non-null   object
 2   TweetAt        3798 non-null   object
 3   OriginalTweet  3798 non-null   object
 4   Sentiment      3798 non-null   object
dtypes: int64(1), object(4)
memory usage: 178.0+ KB
None


In [5]:
print(tweet_whole_train.head())

          ScreenName   Location     TweetAt  \
UserName                                      
3799           48751     London  16-03-2020   
3800           48752         UK  16-03-2020   
3801           48753  Vagabonds  16-03-2020   
3802           48754        NaN  16-03-2020   
3803           48755        NaN  16-03-2020   

                                              OriginalTweet  \
UserName                                                      
3799      @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...   
3800      advice Talk to your neighbours family to excha...   
3801      Coronavirus Australia: Woolworths to give elde...   
3802      My food stock is not the only one which is emp...   
3803      Me, ready to go at supermarket during the #COV...   

                   Sentiment  
UserName                      
3799                 Neutral  
3800                Positive  
3801                Positive  
3802                Positive  
3803      Extremely Negative  


In [6]:
tweet = tweet_whole_train[['OriginalTweet', 'Sentiment']]

#### reference: https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

In [7]:
# lower case all text
tweet['OriginalTweet'] = tweet['OriginalTweet'].str.lower()
#tweet['OriginalTweet'] = tweet_text 
tweet['OriginalTweet'].head(10)

<ipython-input-7-a4d75c87a5e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].str.lower()


UserName
3799    @menyrbie @phil_gahan @chrisitv https://t.co/i...
3800    advice talk to your neighbours family to excha...
3801    coronavirus australia: woolworths to give elde...
3802    my food stock is not the only one which is emp...
3803    me, ready to go at supermarket during the #cov...
3804    as news of the regions first confirmed covid-...
3805    cashier at grocery store was sharing his insig...
3806    was at the supermarket today. didn't buy toile...
3807    due to covid-19 our retail store and classroom...
3808    for corona prevention,we should stop to buy th...
Name: OriginalTweet, dtype: object

In [8]:
stopwordlist = ['a','ain', 'am', 'an',
             'and','any','are', 'as', 'at', 'be',  'before',
             'being', 'between', 'by', 'can', 'd', 'do',
             'does', 'doing',  'during',  'for', 'from',
             'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'in',
             'into','is', 'it', 'its', 'itself', 'll', 'm', 'ma',
             'me', 'my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'or',  'our', 'ours','ourselves',  'own', 're','s', 'she', "shes", 
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 've', 
             'we',  'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [9]:
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: cleaning_stopwords(text))
tweet['OriginalTweet'].head()

<ipython-input-9-6c09ff49c4a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: cleaning_stopwords(text))


UserName
3799    @menyrbie @phil_gahan @chrisitv https://t.co/i...
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia: woolworths give elderly...
3802    food stock not only one empty... please, don't...
3803    me, ready go supermarket #covid19 outbreak. no...
Name: OriginalTweet, dtype: object

In [10]:
def remove_at_signs(text):
    return " ".join([word for word in str(text).split() if not word.startswith('@')])
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: remove_at_signs(text))
tweet['OriginalTweet'].head()

<ipython-input-10-2bef1d3d1219>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda text: remove_at_signs(text))


UserName
3799    https://t.co/ifz9fan2pa https://t.co/xx6ghgfzc...
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia: woolworths give elderly...
3802    food stock not only one empty... please, don't...
3803    me, ready go supermarket #covid19 outbreak. no...
Name: OriginalTweet, dtype: object

In [11]:
def cleaning_URLs(data):
    return re.sub('[^ ]+\.[^ ]+' , ' url', data)
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_URLs(x))
tweet['OriginalTweet'].head()

<ipython-input-11-dbc6b0c6c066>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_URLs(x))


UserName
3799                                        url  url  url
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia: woolworths give elderly...
3802    food stock not only one  url please, don't pan...
3803    me, ready go supermarket #covid19 outbreak. no...
Name: OriginalTweet, dtype: object

In [12]:
tweet.loc[3799, 'OriginalTweet']

' url  url  url'

In [13]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans(' ', ' ', punctuations_list)
    return text.translate(translator)
tweet['OriginalTweet']= tweet['OriginalTweet'].apply(lambda x: cleaning_punctuations(x))
tweet['OriginalTweet'].head()

<ipython-input-13-c0b58da7c31c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet']= tweet['OriginalTweet'].apply(lambda x: cleaning_punctuations(x))


UserName
3799                                        url  url  url
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia woolworths give elderly ...
3802    food stock not only one  url please dont panic...
3803    me ready go supermarket covid19 outbreak not b...
Name: OriginalTweet, dtype: object

In [14]:
import itertools
def cleaning_repeating_char(text):
    cl = ''.join(ch for ch, _ in itertools.groupby(text))
    return cl
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_repeating_char(x))
tweet['OriginalTweet'].head(10)

<ipython-input-14-260ba1a553fd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_repeating_char(x))


UserName
3799                                          url url url
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia wolworths give elderly d...
3802    fod stock not only one url please dont panic e...
3803    me ready go supermarket covid19 outbreak not b...
3804    news regions first confirmed covid19 case cam...
3805    cashier grocery store was sharing insights cov...
3806    was supermarket today didnt buy toilet paper r...
3807    due covid19 retail store clasrom atlanta not o...
3808    corona preventionwe should stop buy things cas...
Name: OriginalTweet, dtype: object

In [15]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_numbers(x))
tweet['OriginalTweet'].head(10)

<ipython-input-15-39d2e1561b65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['OriginalTweet'] = tweet['OriginalTweet'].apply(lambda x: cleaning_numbers(x))


UserName
3799                                          url url url
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia wolworths give elderly d...
3802    fod stock not only one url please dont panic e...
3803    me ready go supermarket covid outbreak not bec...
3804    news regions first confirmed covid case came ...
3805    cashier grocery store was sharing insights cov...
3806    was supermarket today didnt buy toilet paper r...
3807    due covid retail store clasrom atlanta not ope...
3808    corona preventionwe should stop buy things cas...
Name: OriginalTweet, dtype: object

In [16]:
# Tokenize the OriginalTweet text
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
tweet['Tokens'] = [tknzr.tokenize(t) for t in tweet['OriginalTweet']]
tweet['Tokens'].head()

<ipython-input-16-116119311db9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['Tokens'] = [tknzr.tokenize(t) for t in tweet['OriginalTweet']]


UserName
3799                                      [url, url, url]
3800    [advice, talk, neighbours, family, exchange, p...
3801    [coronavirus, australia, wolworths, give, elde...
3802    [fod, stock, not, only, one, url, please, dont...
3803    [me, ready, go, supermarket, covid, outbreak, ...
Name: Tokens, dtype: object

In [17]:
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
tweet['Tokens']= tweet['Tokens'].apply(lambda x: stemming_on_text(x))
tweet['Tokens'].head(10)

<ipython-input-17-9e413b8e77ce>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['Tokens']= tweet['Tokens'].apply(lambda x: stemming_on_text(x))


UserName
3799                                      [url, url, url]
3800    [advice, talk, neighbours, family, exchange, p...
3801    [coronavirus, australia, wolworths, give, elde...
3802    [fod, stock, not, only, one, url, please, dont...
3803    [me, ready, go, supermarket, covid, outbreak, ...
3804    [news, region, , s, first, confirmed, covid, ...
3805    [cashier, grocery, store, was, sharing, insigh...
3806    [was, supermarket, today, didnt, buy, toilet, ...
3807    [due, covid, retail, store, clasrom, atlanta, ...
3808    [corona, preventionwe, should, stop, buy, thin...
Name: Tokens, dtype: object

In [18]:
lm = nltk.stem.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
tweet['Tokens'] = tweet['Tokens'].apply(lambda x: lemmatizer_on_text(x))
tweet['Tokens'].head(10)

<ipython-input-18-da252269bd34>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['Tokens'] = tweet['Tokens'].apply(lambda x: lemmatizer_on_text(x))


UserName
3799                                      [url, url, url]
3800    [advice, talk, neighbours, family, exchange, p...
3801    [coronavirus, australia, wolworths, give, elde...
3802    [fod, stock, not, only, one, url, please, dont...
3803    [me, ready, go, supermarket, covid, outbreak, ...
3804    [news, region, , s, first, confirmed, covid, ...
3805    [cashier, grocery, store, was, sharing, insigh...
3806    [was, supermarket, today, didnt, buy, toilet, ...
3807    [due, covid, retail, store, clasrom, atlanta, ...
3808    [corona, preventionwe, should, stop, buy, thin...
Name: Tokens, dtype: object

In [19]:
tweet['text'] = tweet["Tokens"].map(' '.join)
print(tweet['text'].head(10))

UserName
3799                                          url url url
3800    advice talk neighbours family exchange phone n...
3801    coronavirus australia wolworths give elderly d...
3802    fod stock not only one url please dont panic e...
3803    me ready go supermarket covid outbreak not bec...
3804    news region  s first confirmed covid case cam...
3805    cashier grocery store was sharing insights cov...
3806    was supermarket today didnt buy toilet paper r...
3807    due covid retail store clasrom atlanta not ope...
3808    corona preventionwe should stop buy things cas...
Name: text, dtype: object


<ipython-input-19-cc0ea48d3dc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet['text'] = tweet["Tokens"].map(' '.join)


In [20]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 

X = tweet['text'] 

y = tweet['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df = 10, max_df = 0.7)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(X_test)

# Print the first 10 features
print('# of feature names', len(tfidf_vectorizer.get_feature_names()))
print(tfidf_vectorizer.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train.A[1])

# of feature names 5535
['abc', 'ability', 'able', 'about', 'above', 'absolute', 'absolutely', 'absurd', 'abt', 'abuse']
[0. 0. 0. ... 0. 0. 0.]


In [41]:
pip install xgboost

     |████████████████████████████████| 1.3 MB 2.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [57]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_XG = label_encoder.fit_transform(y_train)
print(y_XG[:5])
y_test_XG = label_encoder.fit_transform(y_test)
print(y_test_XG[:5])

[3 0 4 2 0]
[0 2 4 2 3]


In [59]:
from xgboost import XGBClassifier

XGmodel = XGBClassifier(use_label_encoder=False)
XGmodel.fit(tfidf_train, y_XG)

# make predictions for test data

y_pred_XG = XGmodel.predict(tfidf_test)

# evaluate predictions
print(classification_report(y_test_XG, y_pred_XG))
accuracy = metrics.accuracy_score(y_test_XG, y_pred_XG)
CM_XG = metrics.confusion_matrix(y_test_XG, y_pred_XG)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(CM_XG)

[20:02:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.60      0.49      0.54      1342
           1       0.69      0.51      0.59      1674
           2       0.52      0.46      0.49      2466
           3       0.57      0.72      0.63      1939
           4       0.49      0.56      0.52      2869

    accuracy                           0.55     10290
   macro avg       0.57      0.55      0.55     10290
weighted avg       0.56      0.55      0.55     10290

Accuracy: 54.91%
[[ 655   21  446   91  129]
 [  14  856   81   81  642]
 [ 287   65 1129  428  557]
 [  28   16  131 1391  373]
 [ 114  288  392  456 1619]]


In [60]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [20, 100, 500],
    'colsample_bytree': [0.2, 0.8],
    'max_depth': [10,50],
    'reg_alpha': [1.1, 1.3],
    'reg_lambda': [1.1, 1.3],
    'subsample': [0.5, 0.9]
}

grid_GX = GridSearchCV(XGBClassifier(use_label_encoder=False), param_grid=param_grid, cv = 5, scoring='accuracy', verbose=10)
GX_grid_results = grid_GX.fit(tfidf_train,y_XG)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5; 1/96] START colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.1, subsample=0.5
[20:03:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 1/96] END colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.1, subsample=0.5; total time=   3.6s
[CV 2/5; 1/96] START colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.1, subsample=0.5
[20:04:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavio

[20:04:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 4/96] END colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9; total time=   4.8s
[CV 2/5; 4/96] START colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9
[20:05:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 4/96] END colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9; total time=   4.6s
[CV 3/5; 4/96] START colsample_bytree=0.2

[CV 1/5; 7/96] END colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5; total time=   3.4s
[CV 2/5; 7/96] START colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5
[20:06:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 7/96] END colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5; total time=   3.4s
[CV 3/5; 7/96] START colsample_bytree=0.2, max_depth=10, n_estimators=20, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5
[20:06:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlo

[20:08:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 10/96] END colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.1, reg_lambda=1.1, subsample=0.9; total time=  22.1s
[CV 3/5; 10/96] START colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.1, reg_lambda=1.1, subsample=0.9
[20:09:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 3/5; 10/96] END colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.1, reg_lambda=1.1, subsample=0.9; total time=  22.1s
[CV 4/5; 10/96] START colsample_byt

[CV 2/5; 13/96] END colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.3, reg_lambda=1.1, subsample=0.5; total time=  15.4s
[CV 3/5; 13/96] START colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.3, reg_lambda=1.1, subsample=0.5
[20:14:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 3/5; 13/96] END colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.3, reg_lambda=1.1, subsample=0.5; total time=  15.3s
[CV 4/5; 13/96] START colsample_bytree=0.2, max_depth=10, n_estimators=100, reg_alpha=1.3, reg_lambda=1.1, subsample=0.5
[20:14:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror'

...

[07:06:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 4/5; 84/96] END colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9; total time= 4.5min
[CV 5/5; 84/96] START colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9
[07:10:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 84/96] END colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.1, reg_lambda=1.3, subsample=0.9; total time= 4.5min
[CV 1/5; 85/96] START colsample_byt

[CV 4/5; 87/96] END colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5; total time= 2.7min
[CV 5/5; 87/96] START colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5
[08:01:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 87/96] END colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.3, reg_lambda=1.3, subsample=0.5; total time= 2.7min
[CV 1/5; 88/96] START colsample_bytree=0.8, max_depth=50, n_estimators=100, reg_alpha=1.3, reg_lambda=1.3, subsample=0.9
[08:03:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror'

[10:04:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 90/96] END colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.1, reg_lambda=1.1, subsample=0.9; total time=13.6min
[CV 1/5; 91/96] START colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.1, reg_lambda=1.3, subsample=0.5
[10:18:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 91/96] END colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.1, reg_lambda=1.3, subsample=0.5; total time= 9.0min
[CV 2/5; 91/96] START colsample_byt

[CV 5/5; 93/96] END colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.3, reg_lambda=1.1, subsample=0.5; total time= 8.9min
[CV 1/5; 94/96] START colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.3, reg_lambda=1.1, subsample=0.9
[12:56:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 94/96] END colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.3, reg_lambda=1.1, subsample=0.9; total time=13.3min
[CV 2/5; 94/96] START colsample_bytree=0.8, max_depth=50, n_estimators=500, reg_alpha=1.3, reg_lambda=1.1, subsample=0.9
[13:10:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror'

[16:53:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [62]:
result=pd.DataFrame(grid_GX.cv_results_)
result[result['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_max_depth,param_n_estimators,param_reg_alpha,param_reg_lambda,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
17,107.675748,6.267482,1.251081,0.070695,0.2,10,500,1.1,1.1,0.9,"{'colsample_bytree': 0.2, 'max_depth': 10, 'n_...",0.574668,0.584224,0.576543,0.574113,0.579621,0.577834,0.00373,1
